# PWG 1. 强化学习代码练习

PIG 是 Play With Gym 的缩写，代表这里我们所使用的环境是 OpenAI 的 `gym`。`gym` 为强化学习 AI 提供了许多小游戏。此外我们还需使用 `baselines` 提供的各类 DRL API。利用这两个工具我们就有机会复现 DeepMind 在 2015 年发表的论文 [Human-level control through deep reinforcement learning](https://www.nature.com/articles/nature14236?wm=book_wap_0005)。也就是最经典的让 AI 玩游戏的论文。

> 资料集：
> - [Stable-Baselines3 Docs - Reliable Reinforcement Learning Implementations](https://stable-baselines3.readthedocs.io/en/master/)
> - [Gym](https://gym.openai.com/)
> - [PyTorch](https://pytorch.org/)
> - [Youtube | Nicholas Renotte ](https://www.youtube.com/watch?v=Mut_u40Sqz4&t=1290s&ab_channel=NicholasRenotte) 提供的教程

> 对于 TensorFlow 比较熟悉的同学可以看：
> - [DRL 中文书](https://deepreinforcementlearningbook.org/)

## 1. 准备

安装 `balines` 和 `gym`。在本文中默认使用 Pytorch 作为算法框架，至于如何安装 Pytorch 大家需要根据自己电脑的情况来。

> 这里推荐大家使用 Linux 或 Mac 来使用 Gym，虽然 Windows 也可以使用 Gym，但 Gym 对于 Windows 的支持没有前两者好。

In [ ]:
!pip install stable-baselines3[extra]
!pip install gym[all]

引入各类需要用到的库

In [1]:
import os
import gym
# This is the algorithm that we will use in this tutorial
from stable_baselines3 import PPO
# Train multiple agents at the same time
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## 2. 创建并理解环境

In [2]:
# create the CartPole environment
env = gym.make('CartPole-v0')
episodes = 5
# Loop the env for 5 episodes
for episode in range(1, episodes+1):
    # Initialize the environment and get first state
    state = env.reset()
    done = False
    sorce = 0

    while not done:
        # Render(渲染) the environment
        env.render(mode='rgb_array')
        # Random actions
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        sorce += reward
    print("Episode {}  score: {}".format(episode, sorce))
env.close()

Episode 1  score: 12.0
Episode 2  score: 26.0
Episode 3  score: 12.0
Episode 4  score: 9.0
Episode 5  score: 33.0


![](./pics/WelllitLawfulCero-size_restricted.gif)

现在我们探索一下上面创建环境代码段中每一行的作用，比如 `env.action_space`。这是所有 agent 可以做的动作，及动作空间。

In [8]:
env.action_space

Discrete(2)

其动作分别是：

| Num | Action                 |
|-----|------------------------|
| 0   | Push cart to the left  |
| 1   | Push cart to the right |

`sample` 即采样，我们可以随机的获得一些动作，在这里我们的动作只有 0 或 1.

In [20]:
for i in range(3):
    print(env.action_space.sample())

1
0
1


`env.observation_space` 会返回一个 `box` 这里会告诉我们，系统的状态量的上限，下限，个数，以及数据类型。

In [23]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

这个就解释了`env.reset()`的返回值，`reset()`的中的 4 个值就代表了当前状态的 4 个状态量。

In [24]:
env.reset()

array([0.03442793, 0.04027685, 0.01362426, 0.03519027])

这 4 个状态值分别是：

| Num | Observation           | Min                  | Max                |
|-----|-----------------------|----------------------|--------------------|
| 0   | Cart Position         | -4.8                 | 4.8                |
| 1   | Cart Velocity         | -Inf                 | Inf                |
| 2   | Pole Angle            |  -0.418 rad (-24°)  |  0.418 rad (24°)  |
| 3   | Pole Angular Velocity | -Inf                 |Inf                |

`env.step(1)` 代表的是 Agent 从当前状态执行动作 1 后达到的状态（4 个状态量）以及该状态的 `reward`。`True` 是告诉我们 `episode` 是否结束。

In [4]:
for i in range(3):
    print(env.step(1))

(array([-0.22199357, -0.2106671 ,  0.32290945,  0.72448659]), 0.0, True, {})
(array([-0.22620691, -0.02109515,  0.33739918,  0.54811949]), 0.0, True, {})
(array([-0.22662881,  0.16812675,  0.34836157,  0.37761348]), 0.0, True, {})


## 3. 训练 RL 模型

设置模型的数据储存位置：

In [6]:
log_path = os.path.join("training","logs")
log_path

'training/logs'

下面为训练环境配置硬件设备，和算法。由于我的电脑没有 GPU 所以这里显示的是 `Using cpu device`。但如果你的电脑有 GPU 并且 `cuda` 和 `cuda` 版本的 `PyTorch` 那么这里将会显示 `Using cude device`

In [8]:
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


下面我们来查看一下 PPO 算法该怎么使用，输入 `PPO??` 来查询其参数：

In [9]:
PPO??

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gym.core.Env, stable_baselines3.common.vec_env.base_vec_env.VecEnv, str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    target_kl: Optional[float] = None,
    tensorboard_log: Optional[str] = None,
    create_eval_env: bool = False,
    policy_kwargs: Optional[Dict[str, Any]] = None,
    verbose: int = 0,
    seed: Optional[int] = None,
    device: Union[torch.device, str] = 'auto',
    _init_setup_model: bool = True,
)
Source:        
clas

下面我们来使用 PPO 算法来训练模型。由于 OpenAI 已经把 api 写的非常好了，我们直接用 `model.learn()` 然后设置训练次数就可以了。此外，如果我们多 run 几次下面的 cell，那么每一次 run 都是在原先模型的基础上再次训练。

In [10]:
model.learn(total_timesteps=10000)

Logging to training/logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 3373 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2260        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008283781 |
|    clip_fraction        | 0.0795      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00398    |
|    learning_rate        | 0.0003      |
|    loss                 | 5.69        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0112     |
|    value_loss           | 50.4        |
-----------------------------------------
---

## 4. 储存并再次调用模型

假如我们的训练中断了，需要将模型储存一下方便日后再次训练。那么我们就可以使用下面的代码：

In [11]:
PPO_path = os.path.join("training","Saved_models","PPO")
model.save(PPO_path)

/home/tim/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'training/Saved_models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


这个时候，我们就可以 使用 `del model` 删除模型了.

In [12]:
del model

然后，我们重载模型的时候，使用相应的算法 class + load 就可以调用了，比如 `PPO.load(PPO_path, env=env)`。然后我们需要输入所保存的模型的路径并且设在环境。这里的环境之前以及设置过了，所以直接使用 `env=env` 就可以了。

In [14]:
model = PPO.load(PPO_path, env=env)
model.learn(total_timesteps=10000)

Logging to training/logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 3744 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 2369         |
|    iterations           | 2            |
|    time_elapsed         | 1            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0059628515 |
|    clip_fraction        | 0.0482       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.576       |
|    explained_variance   | 0.529        |
|    learning_rate        | 0.0003       |
|    loss                 | 9.12         |
|    n_updates            | 60           |
|    policy_gradient_loss | -0.00819     |
|    value_loss           | 53.7         |
----------------------------

## 5. 测试与评估模型

下面我们来检验一下我们模型训练的效果：

In [15]:
evaluate_policy(model, env, n_eval_episodes=10,render=True)
env.close()

/home/tim/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

现在我们的模型已经可以做到稳定的控制木杆直立了：

![](./pics/1_oMSg2_mKguAGKy1C64UFlw.gif)

`evaluate_policy()` 本质上和我们最开始写的环境可视化代码做到事情差不多。因此我们将之前的那段代码复制下来。然后将随机选取动作改成让模型来根据当前的状态预测动作以及回到达的下一个状态。其中我们重点修改的部分使用了 `action, _ = model.predict(obs)`。

In [17]:
episodes = 5
# Loop the env for 5 episodes
for episode in range(5):
    # Initialize the environment and get first state
    obs = env.reset()
    done = False
    sorce = 0

    while not done:
        # Render(渲染) the environment
        env.render(mode='rgb_array')
        # Random actions
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        sorce += reward
    print("Episode {}  score: {}".format(episode, sorce))
env.close()

Episode 0  score: [200.]
Episode 1  score: [200.]
Episode 2  score: [200.]
Episode 3  score: [200.]
Episode 4  score: [200.]


这时我们发现，每一次测试的得分都是 200 分了，这说明使用 PPO 算法的训练是非常有效果的。

之前在设置算法参数的时候，我们设置了 `Tensorboard` 选项，他可以用来将算法评估可视化。但这里使用的其实是打开 `Tensorboard` 的命令，这个命令会持续运行，所以最好在一个终端中使用这个命令。

In [20]:
train_path = os.path.join(log_path,"PPO_2")
!tensorboard --logdir=${train_path}

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


从 `http://localhost:6006/` 链接进入 `Tensorboard` 网页后可以看到很多指标的表格数据。网页大概是这样的：

![tensorboard](./pics/tensorboard.png)